In [8]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
file_path = "iowa_data.csv"

home_data = pd.read_csv(file_path)

home_data.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [31]:
y = home_data.SalePrice

feature_names = ["LotArea", "YearBuilt", "1stFlrSF", "2ndFlrSF", "FullBath", "BedroomAbvGr", "TotRmsAbvGrd"]

X = home_data[feature_names]

train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

In [10]:
iowa_home_model = DecisionTreeRegressor(random_state=1)

iowa_home_model.fit(train_X, train_y)

DecisionTreeRegressor(random_state=1)

In [13]:
predicitions = iowa_home_model.predict(val_X)

print("Previsões realizadas: ", predicitons[:5])
print("Preço das casas de entrada como teste: ", val_y.head())

Previsões realizadas:  [208500. 176432. 239500. 200000. 135000.]
Preço das casas de entrada como teste:  536    188000
236    185500
244    205000
941    214000
394    109000
Name: SalePrice, dtype: int64


In [27]:
from sklearn.metrics import mean_absolute_error

val_mae = mean_absolute_error(val_y, predicitions)

print("Validation MAE: {:,.0f}".format(val_mae))

Validation MAE: 32,929


In [22]:
# função que irá medir o MAE (Mean Absolute Error) o número de nós number_leaf_nodes da árvore de decisão

def get_mae(number_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=number_leaf_nodes, random_state=1)
    model.fit(train_X, train_y)
    predict = model.predict(val_X)
    return mean_absolute_error(val_y, predict)


# medindo entre vários candidatos para número de leaf nodes da árvore de decisão qual irá gerar o menor erro absoluto médio (MAE)

candidate_max_leaf_nodes = [5, 25, 50, 100, 250, 500]

scores = {number_leaf_nodes: get_mae(number_leaf_nodes, train_X, val_X, train_y, val_y) for number_leaf_nodes in candidate_max_leaf_nodes}

best_tree_size = min(scores, key=scores.get)

25


In [23]:
# gerando um novo modelo utilizando o número de nós que irá proporcionar o menor MAE

iowa_home_model = DecisionTreeRegressor(max_leaf_nodes=best_tree_size, random_state=1)

iowa_home_model.fit(X, y)

DecisionTreeRegressor(max_leaf_nodes=25, random_state=1)

In [33]:
# usando random forest para treinar um novo modelo

from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(random_state=1)

rf_model.fit(train_X, train_y)

rf_val_mae = mean_absolute_error(val_y, rf_model.predict(val_X))

print("Validation MAE for Random Forest Model: {:,.2f}".format(rf_val_mae))

Validation MAE for Random Forest Model: 21,857.16
